<a href="https://colab.research.google.com/github/sonakshisharan/Machine_Learning/blob/main/PSO_and_ACO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip drive/MyDrive/Interpolated\ Data-10K\ DATA\ POINTS\ -\ 4d,4e,6b.zip

Archive:  drive/MyDrive/Interpolated Data-10K DATA POINTS - 4d,4e,6b.zip
  inflating: Interpolated Data/4d/interpolated_blue4d_6L.csv  
  inflating: Interpolated Data/4d/interpolated_red4d_12L.csv  
  inflating: Interpolated Data/4d/interpolated_black4d_18L.csv  
  inflating: Interpolated Data/4d/interpolated_green4d_3L.csv  
  inflating: Interpolated Data/6b/interpolated_black6b_Ra-ct charge.csv  
  inflating: Interpolated Data/6b/interpolated_blue6b_R0 discharge.csv  
  inflating: Interpolated Data/6b/interpolated_red6b_Ra-ct discharge.csv  
  inflating: Interpolated Data/6b/interpolated_green6b_R0 charge.csv  
  inflating: Interpolated Data/4e/interpolated_red4e_12L.csv  
  inflating: Interpolated Data/4e/interpolated_green4e_3L.csv  
  inflating: Interpolated Data/4e/interpolated_blue4e_6L.csv  
  inflating: Interpolated Data/4e/interpolated_black4e_18L.csv  


In [ ]:
# PSO Algorithm
import numpy as np

# VRFB system parameters
num_cells = 20
power_capacity = 1000  # 1 kW
energy_capacity = 6000  # 6 kWh
voltage_range = (20, 32)  # V
max_current = 60  # A
flow_rate_range = (3, 18)  # L/min

# PSO parameters
num_particles = 50
num_iterations = 100
c1 = 1.5
c2 = 1.6
w = 0.5

# Initialize particles
particles = np.random.uniform(flow_rate_range[0], flow_rate_range[1], (num_particles, 2))
velocities = np.zeros_like(particles)

# Objective function
def objective_function(x, y, stack_current, SOC):
    flow_rate = np.sqrt(x**2 + y**2)  # Calculate flow rate from x and y coordinates
    # Calculate internal resistance, pump power, and system power loss
    #for the flow rate = 3L/min
    Rint = 0.126
    Ppump = 100
    Ploss_VRFB = stack_current**2 * Rint + Ppump
    return Ploss_VRFB

# PSO loop
pbest = np.inf * np.ones((num_particles, 2))
gbest = np.inf
for iteration in range(num_iterations):
    for i in range(num_particles):
        x, y = particles[i, 0], particles[i, 1]
        stack_current = 45
        SOC<=0.1

        Ploss = objective_function(x, y, stack_current, SOC)

        if Ploss < pbest[i].sum():
            pbest[i] = np.array([x, y])

        if Ploss < gbest:
            gbest = Ploss
            optimal_x, optimal_y = x, y

    # Update particle velocities and positions
    velocities = w * velocities + c1 * np.random.rand(2) * (pbest - particles) + c2 * np.random.rand(2) * (gbest - particles)
    particles = particles + velocities

    # Enforce boundary constraints
    particles[:, 0] = np.clip(particles[:, 0], flow_rate_range[0], flow_rate_range[1])
    particles[:, 1] = np.clip(particles[:, 1], flow_rate_range[0], flow_rate_range[1])

print(f"Optimal flow rate coordinates: ({optimal_x}, {optimal_y})")
print(f"Minimum VRFB system power loss: {gbest} W")

# ACO Algorithm
import numpy as np

# VRFB system parameters
num_cells = 20
power_capacity = 1000  # 1 kW
energy_capacity = 6000  # 6 kWh
voltage_range = (20, 32)  # V
max_current = 60  # A
flow_rate_range = (3, 18)  # L/min

# ACO parameters
num_ants = 50
num_iterations = 100
alpha = 1  # Pheromone weight
beta = 2  # Heuristic weight
rho = 0.5  # Evaporation rate
Q = 1  # Constant for pheromone update

# Initialize pheromone matrix
pheromone = np.ones((len(flow_rate_range), len(flow_rate_range), len(flow_rate_range), len(flow_rate_range)))

# Objective function
def objective_function(x, y, stack_current, SOC):
    flow_rate = np.sqrt(x**2 + y**2)  # Calculate flow rate from x and y coordinates
    # Calculate internal resistance, pump power, and system power loss
    Rint = 0.126
    Ppump = 100
    Ploss_VRFB = stack_current**2 * Rint + Ppump
    return Ploss_VRFB



    # ACO loop
best_solution = np.inf
for iteration in range(num_iterations):
    ant_solutions = []
    for ant in range(num_ants):
        X = np.random.choice(range(*flow_rate_range))  # Assuming X represents the flow rate
        Y =   y = np.random.choice(range(*flow_rate_range))
        stack_current =45
        SOC = 0.1

        Ploss = objective_function(X, Y,stack_current, SOC)
        ant_solutions.append(((X, Y), Ploss))  # Store (X, Y) as a tuple

        if Ploss < best_solution:
            best_solution = Ploss
            optimal_X = X

    # Update pheromone matrix
for ant_solution in ant_solutions:
        (x, y), Ploss = ant_solution

        # Ensure indices x and y fall within the bounds of the pheromone array
        x_idx = min(max(x, 0), pheromone.shape[0] - 1)
        y_idx = min(max(y, 0), pheromone.shape[1] - 1)

        pheromone[x_idx, y_idx] += Q / Ploss  # Update the pheromone at position (x, y)

    # Evaporate pheromone
pheromone *= (1 - rho)
#ACO implementation at 3L/min
print(f"Optimal X (flow rate): {optimal_X} L/min")
print(f"Minimum VRFB system power loss: {best_solution} W")


print(f"Optimal flow rate coordinates: ({optimal_x}, {optimal_y})")
print(f"Minimum VRFB system power loss: {best_solution} W")

Optimal flow rate coordinates: (14.226854179258373, 6.894533734468228)
Minimum VRFB system power loss: 355.15 W
Optimal X (flow rate): 6 L/min
Minimum VRFB system power loss: 355.15 W
Optimal flow rate coordinates: (14.226854179258373, 6.894533734468228)
Minimum VRFB system power loss: 355.15 W
